In [64]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Code Example: Linked list v.s. array

In [51]:
render_code("linked_list/linked_list.c")

// linked_list/linked_list.c:1-80 (80 lines)
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "perfstats.h"
struct node
{
    int value;
    int linked;
    struct node *next;
};

int depth_of_list(struct node *start)
{
    int number_of_nodes = 0;
    struct node *node = start;
    while(node)
    {
        node = node->next;
        number_of_nodes++;
    }
    return number_of_nodes;
}

int sum_of_array(struct node *start, int size)
{
    int number_of_nodes = 0, i;
    unsigned long sum=0;
    struct node *node = start;
    for(i=0;i<size;i++)
    {
        if(node[i].next)
            sum+=node[i].value;
    }
    return sum;
}

int main(int argc, char **argv)
{
    unsigned array_size = 131072, depth;
    struct node *data;
    struct timeval time_start, time_end;
        char preamble[1024];
        char epilogue[1024];
        char header[1024];
        char stat_file[] = "stats.csv";
    int list = 0, toLink;
    if(argc >= 2)
        array_size = atoi(argv[1]);
    if(argc >= 3)
        list = atoi(argv[2]);
    data = (struct node*)calloc(array_size, sizeof(struct node));
    for (unsigned i = 0; i < array_size; ++i)
        data[i].value = rand();
    for (unsigned i = 0; i < array_size-1; ++i)
    {
        do {
        toLink = rand() & (array_size-1);}
        while(data[toLink].linked != 0);
        data[i].next = &data[toLink];
        data[toLink].linked = 1;
    }
   gettimeofday(&time_start, NULL);
    perfstats_init();
    perfstats_enable(1);
    if(list)
        depth = depth_of_list(data);
    else
        depth = sum_of_array(data, array_size);
        
   gettimeofday(&time_end, NULL);
    perfstats_disable(1);
    sprintf(epilogue,"\n");
    sprintf(preamble,"");
    perfstats_print(preamble, stat_file, epilogue);
    perfstats_deinit();
   fprintf(stderr, "depth: %lu\n", depth);
//   fprintf(stderr, "sorted %lf seconds\n",((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
   return 0;
}

In [ ]:
! make -C linked_list clean ; make -C linked_list
! echo "size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "4096,0," >> stats.csv
! cs203 job run "linked_list/linked_list 4096 0"
! echo -n "4096,1," >> stats.csv
! cs203 job run "linked_list/linked_list 4096 1"

In [ ]:
display_df_mono(render_csv("stats.csv"))

# Code example: popcount

Given a 64-bit integer number, find the number of 1s in its binary representation.
Example 1:
Input: 9487
Output: 7

Explanation: 9487’s binary representation is 0b10010100001111

In [6]:
render_code("popcounts/popcount.cpp", show=["#else //A", "#endif"])

// popcounts/popcount.cpp:80-90 (11 lines)
#else //A
inline int popcount(uint64_t x){
	int c=0;
	while(x) //while any bits are set not sure how you are getting the value
    {
       c += x & 1;
       x = x >> 1;
    }
    return c;
}
#endif

In [8]:
render_code("popcounts/popcount.cpp", show=["#ifdef B", "#else"])

// popcounts/popcount.cpp:25-41 (17 lines)
#ifdef B
inline int popcount(uint64_t x) {
     int c = 0;
	 while(x) //while any bits are set not sure how you are getting the value
     {
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
     }
     return c;
}
#else

In [9]:
render_code("popcounts/popcount.cpp", show=["#ifdef C", "#else"])

// popcounts/popcount.cpp:13-24 (12 lines)
#ifdef C
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     while(x) {
	       c += table[(x & 0xF)];
	       x = x >> 4;
     }
     return c;
}

#else

In [10]:
render_code("popcounts/popcount.cpp", show=["#ifdef D", "#else"])

// popcounts/popcount.cpp:42-52 (11 lines)
#ifdef D
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     for (uint64_t i = 0; i < 16; i++) {
	       c += table[(x & 0xF)];
	       x = x >> 4;
     }
     return c;
}
#else

In [11]:
render_code("popcounts/popcount.cpp", show=["#ifdef E", "#else"])

// popcounts/popcount.cpp:53-80 (28 lines)
#ifdef E
inline int popcount(uint64_t x) {
     int c = 0;
     for (uint64_t i = 0; i < 16; i++) {
         switch((x & 0xF))
         {
             case 1: c+=1; break;
             case 2: c+=1; break;
             case 3: c+=2; break;
             case 4: c+=1; break;
             case 5: c+=2; break;
             case 6: c+=2; break;
             case 7: c+=3; break;
             case 8: c+=1; break;
             case 9: c+=2; break;
             case 10: c+=2; break;
             case 11: c+=3; break;
             case 12: c+=2; break;
             case 13: c+=3; break;
             case 14: c+=3; break;
             case 15: c+=4; break;
             default: break;
         }
         x = x >> 4;
     }
     return c;
}
#else //A

## Who has the best performance?

In [14]:
render_code("popcounts/popcount.cpp", show="main")

// popcounts/popcount.cpp:107-120 (14 lines)
int main(int argc, char *argv[]) {

     uint64_t key = 0xdeadbeef;

     int count = 1000000000;
     uint64_t sum = 0;
     
     for (int i=0; i < count; i++)
     { 
	   sum += popcount (RandLFSR(key)); 
     }
     printf("Result: %lu\n", sum);
     return sum;
}

In [56]:
! make -C popcounts clean; make -C popcounts
! time ./popcounts/popcount_A
! time ./popcounts/popcount_B
! time ./popcounts/popcount_C
! time ./popcounts/popcount_D
! time ./popcounts/popcount_E

make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
rm -f popcount_A popcount_B popcount_C popcount_D popcount_SSE42 popcount_E
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
g++ -O3 popcount.cpp -o popcount_A
g++ -S -O3 popcount.cpp -o A.s
g++ -O3 -DB popcount.cpp -o popcount_B
g++ -S -O3 -DB popcount.cpp -o B.s
g++ -O3 -DC popcount.cpp -o popcount_C
g++ -S -O3 -DC popcount.cpp -o C.s
g++ -O3 -DD popcount.cpp -o popcount_D
g++ -S -O3 -DD popcount.cpp -o D.s
g++ -O3 -m64 -msse4.2 -DSSE42 popcount.cpp -o popcount_SSE42
g++ -O3 -DE popcount.cpp -o popcount_E
g++ -S -O3 -DE popcount.cpp -o E.s
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
Result: 32000010811

real	0m17.219s
user	0m17.203s
sys	0m0.008s
Result: 32000010811

real	0m14.342s
user	0m14.332s
sys	0m0.000s
Result: 32000010811

re

In [61]:
! perf stat ./popcounts/popcount_C

Result: 32000010811

 Performance counter stats for './popcounts/popcount_C':

          7,618.20 msec task-clock                #    0.999 CPUs utilized          
               764      context-switches          #  100.286 /sec                   
                 3      cpu-migrations            #    0.394 /sec                   
               126      page-faults               #   16.539 /sec                   
    35,250,825,607      cycles                    #    4.627 GHz                      (83.31%)
         4,011,302      stalled-cycles-frontend   #    0.01% frontend cycles idle     (83.31%)
        17,015,493      stalled-cycles-backend    #    0.05% backend cycles idle      (83.33%)
   102,663,837,865      instructions              #    2.91  insn per cycle         
                                                  #    0.00  stalled cycles per insn  (83.36%)
    17,929,856,920      branches                  #    2.354 G/sec                    (83.36%)
        62,661,665   

In [62]:
! perf stat ./popcounts/popcount_D

Result: 32000010811

 Performance counter stats for './popcounts/popcount_D':

          6,617.81 msec task-clock                #    0.999 CPUs utilized          
               666      context-switches          #  100.637 /sec                   
                 6      cpu-migrations            #    0.907 /sec                   
               123      page-faults               #   18.586 /sec                   
    30,610,631,856      cycles                    #    4.625 GHz                      (83.33%)
         3,345,749      stalled-cycles-frontend   #    0.01% frontend cycles idle     (83.33%)
         9,520,378      stalled-cycles-backend    #    0.03% backend cycles idle      (83.32%)
    80,010,362,951      instructions              #    2.61  insn per cycle         
                                                  #    0.00  stalled cycles per insn  (83.33%)
     1,004,068,175      branches                  #  151.722 M/sec                    (83.35%)
           104,052   

In [65]:
! perf stat ./popcounts/popcount_E

Result: 32000010811

 Performance counter stats for './popcounts/popcount_E':

         67,865.52 msec task-clock                #    0.999 CPUs utilized          
             6,823      context-switches          #  100.537 /sec                   
                 4      cpu-migrations            #    0.059 /sec                   
               126      page-faults               #    1.857 /sec                   
   314,512,795,453      cycles                    #    4.634 GHz                      (83.33%)
        33,396,224      stalled-cycles-frontend   #    0.01% frontend cycles idle     (83.33%)
       164,782,309      stalled-cycles-backend    #    0.05% backend cycles idle      (83.33%)
   173,194,383,709      instructions              #    0.55  insn per cycle         
                                                  #    0.00  stalled cycles per insn  (83.33%)
    44,035,633,941      branches                  #  648.866 M/sec                    (83.33%)
    10,401,176,613   

In [66]:
! perf stat ./popcounts/popcount_A

Result: 32000010811

 Performance counter stats for './popcounts/popcount_A':

         17,242.77 msec task-clock                #    0.999 CPUs utilized          
             1,758      context-switches          #  101.956 /sec                   
                 2      cpu-migrations            #    0.116 /sec                   
               127      page-faults               #    7.365 /sec                   
    79,861,728,495      cycles                    #    4.632 GHz                      (83.33%)
         8,516,191      stalled-cycles-frontend   #    0.01% frontend cycles idle     (83.33%)
        41,357,555      stalled-cycles-backend    #    0.05% backend cycles idle      (83.33%)
   332,060,959,098      instructions              #    4.16  insn per cycle         
                                                  #    0.00  stalled cycles per insn  (83.33%)
    65,003,222,325      branches                  #    3.770 G/sec                    (83.35%)
       500,586,652   

## How to implement a branch?


In [23]:
compare([do_render_code("branch/branch.c",show=["#ifdef BITWISE","#else"]),do_render_code("branch/branch.c",show=["#else // B","#endif"]),do_render_code("branch/branch.c",show=["#ifdef NO_BRANCH","#else"]),do_render_code("branch/branch.c",show=["#ifdef COND","#else"])])

In [59]:
! make -C branch clean; make -C branch
! perf stat branch/branch_A 1024 16777216
! perf stat branch/branch_B 1024 16777216
! perf stat branch/branch_C 1024 16777216
! perf stat branch/branch_D 1024 16777216

make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/branch'
rm -f branch_A branch_B branch_A branch_B branch_C branch_D
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/branch'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/branch'
cc -O3  -DHAVE_LINUX_PERF_EVENT_H -g -DBITWISE branch.c -o branch_A
cc -O3  -DHAVE_LINUX_PERF_EVENT_H -g branch.c -o branch_B
cc -O3  -DHAVE_LINUX_PERF_EVENT_H -g -DNO_BRANCH branch.c -o branch_C
cc -O3  -DHAVE_LINUX_PERF_EVENT_H -g -DCOND branch.c -o branch_D
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/branch'
sum: 9921979200
Takes 5.030379 seconds

 Performance counter stats for 'branch/branch_A 1024 16777216':

          5,641.84 msec task-clock                #    0.999 CPUs utilized          
               565      context-switches          #  100.145 /sec                   
                 6      cpu-migrations            #    1.063 /sec 

## ILP of your program

In [58]:
! perf stat ./popcounts/popcount_D

Result: 32000010811

 Performance counter stats for './popcounts/popcount_D':

          6,620.65 msec task-clock                #    0.999 CPUs utilized          
               657      context-switches          #   99.235 /sec                   
                 4      cpu-migrations            #    0.604 /sec                   
               126      page-faults               #   19.031 /sec                   
    30,609,867,746      cycles                    #    4.623 GHz                      (83.33%)
         3,508,903      stalled-cycles-frontend   #    0.01% frontend cycles idle     (83.33%)
         6,738,294      stalled-cycles-backend    #    0.02% backend cycles idle      (83.33%)
    80,003,156,972      instructions              #    2.61  insn per cycle         
                                                  #    0.00  stalled cycles per insn  (83.33%)
     1,004,455,648      branches                  #  151.716 M/sec                    (83.33%)
           105,684   